In [ ]:
from tokenization_utils import *
import json
from tqdm.notebook import tqdm
import jsonlines


# Token Type

In [ ]:
import pickle
program_data_map = "/home/mingzhu/CodeModel/g4g/map_data/"
token_type_path = "/home/mingzhu/CodeModel/CodeGen/token_type/new_codenet_tokenized_data/"
path_token = token_type_path + "token2type.pickle"
path_type = token_type_path + "type2token.pickle"
path_bpe = token_type_path + "token2bpe.pickle"
langs = ["C++", "Java", "Python", "C#", "Javascript", "PHP", "C"]
for lang in langs:
    tokenized_type_path = token_type_path + lang + "/"

def get_programs(lang):
    fn = program_data_map + lang + "-program-tok.jsonl"
    with open(fn) as infile:
        lines = infile.readlines()
    programs = []
    program_tokens = []
    for line in lines:
        dic = json.loads(line.strip())
        program_raw = " ".join(dic['tokens'])
        line = program_raw.replace("@@ ", "")
        program = detok_format(line, file_detokenizers[lang])
        programs.append(program)
        program_tokens.append(dic['tokens'])
    return programs, program_tokens


with open(path_token, 'rb') as infile:
    dic_token = pickle.load(infile)
with open(path_type, 'rb') as infile:
    dic_type = pickle.load(infile)
with open(path_bpe, 'rb') as infile:
    dic_bpe = pickle.load(infile)

lang = "Java"
java_programs, java_program_tokens = get_programs(lang)

In [ ]:
# dict_bpe_token = {lang:[] for lang in langs}
# 判断当前是不是identifier，还要排除string的情况（string的话，上一个token应该是"或者'）
# token2bpe好像有Bug, " Tree ▁ is ▁ foldable " ['"', 'Tree', '▁', 'is', 'fold@@', 'able']， 怎么少了一个“？
dict_bpe_type = {lang:[] for lang in langs}
for lang in langs:
    for token, bpe_list in dic_bpe[lang].items():
        for bpe in bpe_list:
            if token in dic_token[lang]:
                if "\"" in bpe_list:
                    
                if bpe not in dict_bpe_type:
                    dict_bpe_type[bpe] = dic_token[lang][token]
                elif dict_bpe_type[bpe] != dic_token[lang][token]:
                    print("dup", bpe, token, bpe_list, dic_token[lang][token], dict_bpe_type[bpe])

In [ ]:
lang_dic_token = dic_token[lang]
good_identifier_set = set()
line_dic = {i+1:[] for i in range(4)}
for program in java_programs:
#     print(program)
    lines = program.split("\n")
    for line in lines:
        tokens = line.strip().split()
        rule_set = set()
        new_set = set()
        
#         if "=" in tokens or tokens[0] in set(["int", "String"]):
#             if tokens[0] not in set(['return', 'import', 'for', 'if', 'else', "while", 'System', 'tree']):
#                 rule_set.add(line)
        if tokens[0] not in lang_dic_token:
            continue
        elif lang_dic_token[tokens[0]][0] == "identifier":
            bad_cases_identifier = set(['tree', 'System', 'dfs'])
            if tokens[-1] in [';', '{']:
                if tokens[0] not in bad_cases_identifier:
                    if tokens[0][0].isupper():
                        if len(tokens) > 1:
                            if tokens[1] == '.':
                                continue
                        if len(tokens) > 2:
                            if tokens[1] == '[' and tokens[2] != ']':
                                continue
                        good_identifier_set.add(tokens[0])
                        line_dic[1].append(line)
                    elif '=' in tokens:
                        good_identifier_set.add(tokens[0])
                        line_dic[2].append(line)
   
        elif lang_dic_token[tokens[0]][0] == "keyword" :
            bad_cases_keyword = set(['return', 'import', 'for', 'if', 'else', "while"])
            good_cases_keyword = set(['boolean', 'char', 'class', 'double', 
                                      'final', 'float', 'int', 'long', 'new', 
                                      'private', 'public', 'static', 'void', 'this'])
            if tokens[0] in good_cases_keyword:
                if tokens[-1] in [';', '{']:
                    if '=' in tokens:
                        line_dic[3].append(line)
                    else:
                        line_dic[4].append(line)
#         for x in rule_set:
#             if x not in new_set:
#                 print("aaa", x)
        
#     break

In [ ]:
def is_define_java(line, lang_dic_token):
    tokens = line.strip().split()
    flag = False
    if tokens[0] not in lang_dic_token:
        return flag
    elif lang_dic_token[tokens[0]][0] == "identifier":
        bad_cases_identifier = set(['tree', 'System', 'dfs'])
        if tokens[0] not in bad_cases_identifier:
            if tokens[0][0].isupper():
                if len(tokens) > 1:
                    if tokens[1] == '.':
                        return flag
                if len(tokens) > 2:
                    if tokens[1] == '[' and tokens[2] != ']':
                        return flag
                flag = True
    elif lang_dic_token[tokens[0]][0] == "keyword" :
        good_cases_keyword = set(['boolean', 'char', 'class', 'double', 
                                  'final', 'float', 'int', 'long', 'new', 
                                  'private', 'public', 'static', 'void', 'this'])
        if tokens[0] in good_cases_keyword:
            flag = True
    return flag

is_define_java("class BinaryTree ", lang_dic_token)

In [ ]:
lang_dic_token = dic_token[lang]

for program, program_tokens in zip(java_programs, java_program_tokens):
    lines = program.split("\n")
    program_raw_tokens = []
    for line in lines:
        program_raw_tokens.extend(line.split())
        program_raw_tokens.append('\n')
    
    last_line = lines[0]
    line_n = 0
    for token_raw in program_raw_tokens:
        if token_raw == "\n":
            line_n += 1
            if line_n < len(lines) - 1:
                last_line = lines[line_n]
            continue
        if token_raw in dic_bpe[lang]:
            bpe_tokens = dic_bpe[lang][token_raw]
            for i, token in enumerate(bpe_tokens):
                if (token in lang_dic_token and 
                    lang_dic_token[token] == "identifier") or ((token.endswith("@@") 
                                                                    or bpe_tokens[i-1].endswith("@@")) 
                                                                    and "\"" not in last_line
                                                                    and "\'" not in last_line):

#                     print(last_line, token_raw, token)
                    pass
        elif not ("\"" in token_raw or "\'" in token_raw):
            print("Exception", last_line, token_raw)

        # 这里判断identifier可能有些问题，因为string也会有@@
        

In [ ]:
import javalang
tokens = javalang.tokenizer.tokenize('gap == ( gap * 10 );')
parser = javalang.parser.Parser(tokens)
parser.parse_expression()


In [ ]:
key = 2
for line in line_dic[key]:
    print(line)

In [ ]:
import javalang
source = """import java . io . * ;
import java . util . * ;
class CountTriangles {
  static int findNumberOfTriangles ( int arr [ ] ) {
    int n = arr . length ;
    Arrays . sort ( arr ) ;
    int count = 0 ;
    for ( int i = 0 ; i < n - 2 ; ++ i ) {
      int k = i + 2 ;
      for ( int j = i + 1 ; j < n ; ++ j ) {
        while ( k < n && arr [ i ] + arr [ j ] > arr [ k ] ) ++ k ;
        if ( k > j ) count += k - j - 1 ;
      }
    }
    return count ;
  }
  public static void main ( String [ ] args ) {
      ArrayList<String> cars = new ArrayList<String>();
      int arr[] = {
      10 , 21 , 22 , 100 , 101 , 200 , 300 };
      System . out . println ( "Total number of triangles is " + findNumberOfTriangles ( arr ) ) ;
    }
  }"""

source_tokens = []
types = set()
for token in javalang.tokenizer.tokenize(source):
    source_tokens.append((type(token).__name__, token.value))
    types.add(type(token).__name__)
print(source_tokens)